Analyzing trends in vehicle registration times over time.

Analysis of Personal Vehicles for Sustainability

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
# df = pd.read_csv("nc-vehicle-registration.csv",delimiter=";")
# df = pd.read_csv("nc-vehicle-registration.csv",sep=r'\s*;*\s*')
df = pd.read_csv('nc-vehicle-registration.csv', skipinitialspace=True, delimiter=';')

df.columns


In [ ]:
import pandas as pd
import numpy as np

def preprocess_data(df, start_year=2020, end_year=2024):
    """
    Preprocesses the data by handling NaN values, sorting by date, and filtering by year range.
    
    Parameters:
    - df (DataFrame): The input DataFrame containing the raw vehicle data.
    - start_year (int): The start year for filtering data (default is 2020).
    - end_year (int): The end year for filtering data (default is 2024).
    
    Returns:
    - df (DataFrame): The preprocessed DataFrame with interpolated values and sorted by date.
    """
    # Convert the 'Date' column to datetime format
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Filter data based on the specified year range
    df = df[(df['Year'] >= start_year) & (df['Year'] <= end_year)]
    
    # Sort the dataframe by Date and Area Name to ensure chronological order
    df = df.sort_values(by=['Area Name', 'Date'])
    
    # Interpolate missing values for NaN (if any) for vehicle data
    df.interpolate(method='linear', inplace=True)
    
    return df



def calculate_change_and_rate(df):
    # Calculate the change for each vehicle type
    df['EH'] = df["All Hybrids"] + df["Electric"]
    df['Electric Change'] = df.groupby('Area Name')['Electric'].diff()
    df['Hybrids Change'] = df.groupby('Area Name')['All Hybrids'].diff() 
    df['Gas Change'] = df.groupby('Area Name')['Gas'].diff() 
    df['Diesel Change'] = df.groupby('Area Name')['Diesel'].diff() 
    # df["EH Change"] = df.groupby('Area Name')['EH'].diff()




    # Calculate the rate of change as a percentage
    df['Electric Rate of Change'] = (df['Electric Change'] / df.groupby('Area Name')['Electric'].shift(1)) * 100
    df['Hybrids Rate of Change'] = (df['Hybrids Change'] / df.groupby('Area Name')['All Hybrids'].shift(1)) * 100
    df['Gas Rate of Change'] = (df['Gas Change'] / df.groupby('Area Name')['Gas'].shift(1)) * 100
    df['Diesel Rate of Change'] = (df['Diesel Change'] / df.groupby('Area Name')['Diesel'].shift(1)) * 100

    # df['Electric and Hybrids Rate of Change'] = (df['EH Change'] / df.groupby('Area Name')['EH'].shift(1)) * 100

    return df


def export_by_area(df,data_col_graph='Electric Total Change'):
    # Initialize dictionaries for both detailed data and graphical data
    area_data_dict = {}
    graphic_data_dict = {}
    temp_dict = {}
    
    for area_name, group in df.groupby('Area Name'):
        # Filter data for the period 2020-2024
        group_filtered = group[(group['Year'] >= 2020) & (group['Year'] <= 2024)]

        # Export detailed data (same as before)
        area_data_dict[area_name] = group[['Date', 'Electric Change', 'Electric Rate of Change', 
                                           'Hybrids Change', 'Hybrids Rate of Change']].dropna().values.tolist()
        
        # Calculate the total change and average rate of change for the period 2020-2024
        total_electric_change = group_filtered['Electric Change'].sum()
        total_hybrid_change = group_filtered['Hybrids Change'].sum()
        total_gas_change = group_filtered['Gas Change'].sum()
        total_diesel_change = group_filtered['Diesel Change'].sum()

        total_eh_change = total_electric_change + total_hybrid_change
        total_gd_change = total_gas_change + total_diesel_change

        avg_electric_rate_of_change = group_filtered['Electric Rate of Change'].mean()
        avg_hybrid_rate_of_change = group_filtered['Hybrids Rate of Change'].mean()
        avg_gas_rate_of_change = group_filtered['Gas Rate of Change'].mean()
        avg_diesel_rate_of_change = group_filtered['Diesel Rate of Change'].mean()

        avg_all_eh_rate_of_change = avg_electric_rate_of_change + avg_hybrid_rate_of_change
        avg_all_gd_rate_of_change = avg_gas_rate_of_change + avg_diesel_rate_of_change

        total_ed_gd_change = total_eh_change - total_gd_change
        avg_ed_gd_rate_of_change = avg_all_eh_rate_of_change - avg_all_gd_rate_of_change

        # Store the graphical data (summary statistics)
        temp_dict  = {
            'Electric Total Change': total_electric_change,
            'Hybrid Total Change': total_hybrid_change,
            'Electric + Hybrid Total Change': total_eh_change,
            'Electric Avg Rate of Change': avg_electric_rate_of_change,
            'Hybrid Avg Rate of Change': avg_hybrid_rate_of_change,
            'Electric + Hybrid Avg Rate of Change': avg_all_eh_rate_of_change,
            'EH-GD Total Change' : total_ed_gd_change,
            'EH-GD average Rate of Change' : avg_ed_gd_rate_of_change
        }

        graphic_data_dict[area_name] = temp_dict[data_col_graph]
    
    return area_data_dict, graphic_data_dict



## full pipeline to process the vehicle data
def process_vehicle_data(df,start_year=2020,end_year=2024,data_col_graph='Electric Total Change'):
    # Step 1: Preprocess data (handle NaN and sorting)
    df = preprocess_data(df,start_year=start_year,end_year=end_year)
    
    # Step 2: Calculate the change and rate of change
    df = calculate_change_and_rate(df)
    
    # Step 3: Export the data by Area Name in dictionary format
    area_data_dict,graph_data_dict = export_by_area(df,data_col_graph=data_col_graph)    
    return area_data_dict,graph_data_dict


Test the pipeline with the dataframe and then print out the dictionary

now we can write a function that tries to find the change in electric cars over years

we will first interpolate values to fill in missing data

Now we can begin to do a graphical visual here for electric

In [ ]:
import geopandas as gpd

# Load the existing GeoJSON file
gdf = gpd.read_file("north-carolina-geographic-regions.geojson")
# gdf

In [ ]:


# Ensure the data has a 'region_name' field
if 'region_name' not in gdf.columns:
    raise ValueError("GeoJSON file does not contain 'region_name' field.")

# Group by region and dissolve boundaries
gdf_merged = gdf.dissolve(by="fips")

# Save the new GeoJSON file
gdf_merged.to_file("merged_regions.geojson", driver="GeoJSON")

print("New GeoJSON file with merged counties by region is created!")


Now we create a folium map overlayed with these counties borders now

In [ ]:
import geopandas as gpd
import folium
import pandas as pd
import matplotlib.colors as mcolors

# Load NC counties shapefile from GeoPandas datasets or external source
nc_counties = gpd.read_file("merged_regions.geojson")
# nc_counties

Read the county borders GeoJson file

In [ ]:
import geopandas as gpd

world = gpd.read_file('merged_regions.geojson')

In [ ]:
import json

def create_graphic_df(change_dict,data_col='Electric Total Change'):
    df_graphic = pd.DataFrame.from_dict(change_dict, orient='index')



    # Reset index to make 'Area Name' a column
    df_graphic.reset_index(inplace=True)
    df_graphic.columns=['county',data_col]


    return df_graphic



## now remove all except for Area name and specified field __
def drop_cols_all_but(df,data_col='Electric Total Change'):

    columns_to_keep = ['county', data_col]
    df_selected = df[columns_to_keep]
    return df_selected


def merge_geojson(changes_df, data_col='Electric Total Change', save_path="updated_geojson.geojson"):

    #   Merge GeoDataFrame with    changes DataFrame
    merged = world.merge(changes_df[['county', data_col]], how='left', on='county')

    merged["value"] = merged[data_col]

    updated_geojson_path = "updated_geojson.geojson"
    merged.to_file(updated_geojson_path, driver="GeoJSON")

    print("GeoJSON file updated successfully.")
    return merged


def merge_geojson_pipeline(change_dict, data_col='Electric Total Change'):

    df_g = create_graphic_df(change_dict=change_dict, data_col=data_col)

    df_select = drop_cols_all_but(df_g,data_col=data_col)

    return merge_geojson(df_select, data_col=data_col)






In [ ]:
# simple change in electric

delta_dict_electric, delta_graph_dict_electric = process_vehicle_data(df,data_col_graph='Electric Total Change')
merged_electric = merge_geojson_pipeline(change_dict=delta_graph_dict_electric,data_col='Electric Total Change')

# simple change in hybrid
delta_dict_hybrid, delta_graph_dict_hybrid = process_vehicle_data(df,data_col_graph='Hybrid Total Change')
merged_hybrid = merge_geojson_pipeline(change_dict=delta_graph_dict_hybrid,data_col='Hybrid Total Change')

#simple change in hybrid and electric
delta_dict_combo, delta_graph_dict_combo = process_vehicle_data(df,data_col_graph='Electric + Hybrid Total Change')
merged_combo = merge_geojson_pipeline(change_dict=delta_graph_dict_combo,data_col='Electric + Hybrid Total Change')

# average rate of change in electric
delta_dict_rate_electric, delta_graph_dict_rate_electric = process_vehicle_data(df,data_col_graph='Electric Avg Rate of Change')
merged_rate_electric = merge_geojson_pipeline(change_dict=delta_graph_dict_rate_electric,data_col='Electric Avg Rate of Change')

# average rate of change in hybrids
delta_dict_rate_hybrid, delta_graph_dict_rate_hybrid = process_vehicle_data(df,data_col_graph='Hybrid Avg Rate of Change')
merged_rate_hybrid = merge_geojson_pipeline(change_dict=delta_graph_dict_rate_hybrid,data_col='Hybrid Avg Rate of Change')

#average rate of change in electric + hybrid
delta_dict_rate_combo, delta_graph_dict_rate_combo = process_vehicle_data(df,data_col_graph='Electric + Hybrid Avg Rate of Change')
merged_rate_combo = merge_geojson_pipeline(change_dict=delta_graph_dict_rate_combo,data_col='Electric + Hybrid Avg Rate of Change')



## seperate image, doing eh-gd
# total diff
delta_dict_EHGD, delta_graph_dict_EHGD = process_vehicle_data(df,data_col_graph='EH-GD Total Change')
merged_EHGD = merge_geojson_pipeline(change_dict=delta_graph_dict_EHGD,data_col='Electric + Hybrid Avg Rate of Change')

#average rate of change diff
delta_dict_rate_EHGD, delta_graph_dict_rate_EHGD = process_vehicle_data(df,data_col_graph='Electric + Hybrid Avg Rate of Change')
merged_rate_EHGD = merge_geojson_pipeline(change_dict=delta_graph_dict_rate_EHGD,data_col='Electric + Hybrid Avg Rate of Change')


delta_graph_dict_rate_EHGD

Attempt to plot borders

In [ ]:
print(delta_graph_dict_hybrid['Jones County'])
print(delta_graph_dict_combo['Jones County'])
print(delta_graph_dict_electric['Jones County'])


delta_graph_dict_electric['Jones County'] + delta_graph_dict_hybrid['Jones County'] - delta_graph_dict_combo['Jones County']

Final Version:

In [ ]:
import folium
import branca
import numpy as np
import geopandas as gpd
from shapely.geometry import shape
from typing import Dict, Union
from branca.element import MacroElement
from jinja2 import Template


# Define style function
def style_function(feature,dictionary,min,max,color_map):
    county = feature['properties']['county']
    value = next((v for k, v in dictionary.items() if county in k), None)

    # If value is NaN or None, return gray
    if value is None or (isinstance(value, float) and np.isnan(value)):
        return {
            'fillColor': '#AAAAAA',  # Light gray for missing values
            'weight': 1.5,
            'color': 'black',
            'opacity': 1,
            'fillOpacity': 1
        }

    # If value is below -2, return the color for -2 (lower bound)
    if value < min:
        return {
            'fillColor': color_map(min),
            'weight': 1.5,
            'color': 'black',
            'opacity': 1,
            'fillOpacity': 1
        }

    # If value is above 2, return the color for 2 (upper bound)
    if value > max:
        return {
            'fillColor': color_map(max),
            'weight': 1.5,
            'color': 'black',
            'opacity': 1,
            'fillOpacity': 1
        }

    # For values within the range, apply the colormap
    return {
        'fillColor': color_map(value),
        'weight': 1.5,
        'color': 'black',
        'opacity': 1,
        'fillOpacity': 1
    }

def create_nc_map(min: int, max: int, dictionary: Dict[str, float], merged_map: Union[gpd.GeoDataFrame, dict], color_map: branca.colormap.LinearColormap, colormap_caption: str) -> folium.Map:
    """
    Generates a folium map of North Carolina counties, color-coded by values provided in a dictionary.

    This function creates an interactive choropleth map using Folium, overlaying county-level data
    on a base map of North Carolina. It supports custom color maps and automatically adds a color legend.

    Parameters
    ----------
    dictionary : dict
        A dictionary where keys are strings (usually county names or identifiers) and values are numerical
        data to be visualized (e.g., rate of change, percentage growth).

    merged_map : GeoDataFrame or dict
        A GeoPandas GeoDataFrame (or equivalent GeoJSON dictionary) containing the geometries for
        North Carolina counties. Must include a `"county"` field in each feature's properties.

    color_map : branca.colormap.LinearColormap
        A branca color map used to color the counties based on the dictionary values.
        The colormap should already be scaled appropriately to the data range.

    colormap_caption : str
        The caption to display under the color legend on the map (e.g., "Rate of Change in EV Registrations").

    Returns
    -------
    folium.Map
        An interactive Folium map object displaying North Carolina with counties color-coded by data values.

    Raises
    ------
    ValueError
        If `dictionary` is empty or contains no valid numeric values.

    Notes
    -----
    - The function assumes that county names in the GeoJSON match keys in the `dictionary`.
    - Features with no matching data, NaN values, or values of 0 are excluded from label overlays.
    - The `style_function` must be defined elsewhere in your code. It is used to color each county feature.
    - This function does not display value labels by default unless you add a text overlay feature.
    """
    # Ensure changes is not empty
    if not dictionary:
        raise ValueError("Error: 'changes' dictionary is empty!")

    # Convert values to a list and filter out NaN/non-numeric values
    values = [v for v in dictionary.values() if isinstance(v, (int, float)) and not np.isnan(v)]

    # If no valid numeric values exist, set a default range
    if not values:
        raise ValueError("Error: No valid numeric values in 'changes'!")


    # Create the colormap, ensuring that 0 corresponds to white
    colormap = color_map
    
    # Set caption for the colormap
    colormap.caption = colormap_caption

    # Convert GeoDataFrame to GeoJSON if needed
    if isinstance(merged_map, gpd.GeoDataFrame):
        merged_map = merged_map.__geo_interface__  # Convert to GeoJSON format

    # Create folium map
    m = folium.Map(location=[35.5, -79.5], zoom_start=7.3, tiles="Stamen Toner")


    # Instead of adding the colormap directly, extract its HTML
    colormap_html = colormap._repr_html_()
    
    # Use a MacroElement with fixed positioning to force the legend at the bottom center
    custom_template = Template("""
    {% macro html(this, kwargs) %}
    <div id="custom-colormap" style="
         position: fixed;
         bottom: 10px;
         left: 50%;
         transform: translateX(-50%) scale({{ this.colormap_scale }});
         z-index: 9999;
         background-color: white;
         padding: 5px;
         border: 1px solid rgba(0,0,0,0.2);
         border-radius: 4px;
         box-shadow: 0 0 15px rgba(0,0,0,0.2);
         ">
      {{ this.colormap_html | safe }}
    </div>
    {% endmacro %}
    """)
    
    custom_colormap = MacroElement()
    custom_colormap._template = custom_template
    custom_colormap.colormap_html = colormap_html
    custom_colormap.colormap_scale = 1.5
    m.get_root().add_child(custom_colormap)



    # Add GeoJSON layer
    geojson_layer = folium.GeoJson(
        merged_map,
        style_function= lambda feature: style_function(feature,dictionary=dictionary, min=min,max=max,color_map=colormap)
    )
    geojson_layer.add_to(m)




   

    filepath = f"{colormap_caption}.html"
    m.save(filepath)
    return filepath
    # Display the map
    # m.save("nc_map.png")

# branca.colormap.linear.RdBu_05.scale(min_ratio, max_ratio)
caption_end = 'Vehicles Registered 2020-2024.'

m1_path = create_nc_map(-2500,2500, delta_graph_dict_electric,merged_electric,branca.colormap.linear.RdBu_05.scale(-2500,2500),f"Total Change in Electric {caption_end}")
m2_path = create_nc_map(-2500,2500, delta_graph_dict_hybrid,merged_hybrid,branca.colormap.linear.RdBu_05.scale(-2500,2500),f"Total Change in Hybrid {caption_end}")
m3_path = create_nc_map(-2500,2500, delta_graph_dict_combo,merged_combo,branca.colormap.linear.RdBu_05.scale(-2500,2500),f"Total Change in Electric + Hybrid {caption_end}")


m4_path = create_nc_map(-5,5, delta_graph_dict_rate_electric,merged_rate_electric,branca.colormap.linear.BrBG_09.scale(-5,5).to_step(40),f"Average Rate of Change in Electric {caption_end}")
m5_path = create_nc_map(-5,5, delta_graph_dict_rate_hybrid,merged_rate_hybrid,branca.colormap.linear.BrBG_09.scale(-5,5).to_step(40),f"Average Rate of Change in Hybrid {caption_end}")
m6_path = create_nc_map(-5,5, delta_graph_dict_rate_combo,merged_rate_combo,branca.colormap.linear.BrBG_09.scale(-5,5).to_step(40),f"Average Rate of Change in Electric + Hybrid {caption_end}")


edgh_path_1 = create_nc_map(-10000,10000,delta_graph_dict_EHGD,merged_EHGD,branca.colormap.linear.BrBG_09.scale(-10000,10000),f"E+H v.s. G+D Total Difference {caption_end}")
edgh_path_2 = create_nc_map(-10,10,delta_graph_dict_rate_EHGD,merged_rate_EHGD, branca.colormap.linear.BrBG_09.scale(-10,10).to_step(40),f"E+H v.s. G+D Average Rate of Change Difference {caption_end}")


Save the graph

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from PIL import Image

folder_path = './'
# folder_path = 'file://'





In [ ]:
import branca.colormap as cm
from PIL import Image
    

def take_screenshot(map_filepath,png_path):

    # Configure headless Chrome
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")

    # Set your desired window size (adjust as needed)
    driver = webdriver.Chrome(options=chrome_options)
    driver.set_window_size(1024, 768)

    driver.get(folder_path + map_filepath)

    # Wait a couple of seconds to ensure the map fully loads
    time.sleep(2)

    # Save a screenshot
    driver.save_screenshot(png_path)
    driver.quit()

    img = Image.open(png_path)

    left = 100 
    top = 100    
    right = img.width - 150  
    bottom = img.height  

    # Crop the image
    cropped_img = img.crop((left, top, right, bottom))

    # Save the final image
    cropped_img.save(png_path)

    print(f"Map saved.")
   

def combine_images_horizontally(image_paths, output_path, spacing=30, bg_color=(255, 255, 255)):
    images = [Image.open(path) for path in image_paths]
    widths, heights = zip(*(img.size for img in images))
    
    total_width = sum(widths) + spacing * (len(images) - 1)
    max_height = max(heights)
    
    new_img = Image.new('RGB', (total_width, max_height), color=bg_color)
    
    x_offset = 0
    for i, img in enumerate(images):
        new_img.paste(img, (x_offset, 0))
        x_offset += img.width + spacing
    
    new_img.save(output_path)

from PIL import Image

def stack_images_vertically(image_paths, output_path, padding=30, bg_color=(255, 255, 255)):
    """
    Stacks multiple images vertically with specified padding between them.
    
    Parameters:
        image_paths (list of str): List of file paths to the images.
        output_path (str): File path for the resulting stacked image.
        padding (int): The vertical padding (in pixels) to add between images.
        bg_color (tuple): The background color (R, G, B) for the canvas.
        
    Returns:
        None: The function saves the combined image to output_path.
    """
    images = [Image.open(path) for path in image_paths]
    max_width = max(img.width for img in images)
    total_height = sum(img.height for img in images) + padding * (len(images) - 1)
    combined_img = Image.new("RGB", (max_width, total_height), color=bg_color)
    
    y_offset = 0
    for img in images:
        x_offset = (max_width - img.width) // 2  
        combined_img.paste(img, (x_offset, y_offset))
        y_offset += img.height + padding
    
    
    combined_img.save(output_path)


def big_map_comparisons():
    
    paths = []
    paths.append(m1_path)
    paths.append(m2_path)
    paths.append(m3_path)

  
    png_paths = []
    for html_path in paths:
        png_path = html_path.replace(".html", ".png")
        take_screenshot(html_path,png_path)
        png_paths.append(png_path)


    combine_images_horizontally(png_paths, "final_map_combo1.png")


    paths2 = []
    paths2.append(m4_path)
    paths2.append(m5_path)
    paths2.append(m6_path)

    png_paths = []
    for html_path in paths2:
        png_path = html_path.replace(".html", ".png")
        take_screenshot(html_path,png_path)
        png_paths.append(png_path)

    combine_images_horizontally(png_paths, "final_map_combo2.png")

    stack_images_vertically(['final_map_combo1.png', 'final_map_combo2.png'], 'maps_final.png')



paths3 = []
paths3.append(edgh_path_1)
paths3.append(edgh_path_2)

png_paths = []
for html_path in paths3:
    png_path = html_path.replace(".html", ".png")
    take_screenshot(html_path,png_path)
    png_paths.append(png_path)

combine_images_horizontally(png_paths, "differences_map.png")
